In [1]:
#Creating the app.py through jupyter. For my benefit. Will convert later.

#Not all changes in the app.py will be reflected here. I learned the hard way jupyter notebook doesnt play nice with flask. So its better to completely ignore this file unless someone is curious to see me debugging. 

#import Flask, jsonify from the module flask
from flask import Flask, jsonify

#import every module and funtions i may need to accomplish this
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from dateutil.relativedelta import relativedelta


In [2]:
#prepare the engine and the reflection. 

engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)


In [3]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [4]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [5]:
# Lets create the flask server

ClimateApp = Flask(__name__)


In [6]:
# Creating the routes. Lets create the first route
@ClimateApp.route("/")
def welcome():
    """List all routes that are available"""
    return (
        f"Welcome to the Giant Robot Climate App<br/>"
        f"Giant Robots protects these routes</br>"
        f"/api/v1.0/precipitation</br>"
        f"/api/v1.0/stations</br>"
        f"/api/v1.0/tobs</br>"
        f"/api/v1.0/<start></br>"
        f"/api/v1.0/<end></br>"
    )

In [8]:
#create the precipation route
# start a session. 
session = Session(engine)


# Convert the query results to a dictionary using `date` as the key and `prcp` as the value.
# Return the JSON representation of your dictionary.
   


Pdata = []
#@ClimateApp.route("/api/v1.0/precipitation")
def precipitation():
   #Copying what i did from the starter
    twelvemonths = dt.date(2017, 8, 23) - relativedelta(years=1)
    results = session.query(measurement.date,  measurement.prcp).\
       filter(measurement.date >= twelvemonths).order_by(measurement.date.asc()).all()

    for date, prcp in results:
        pdict = {}
        pdict[date] = prcp 
        Pdata.append(pdict)
    #return jsonify(Pdata)
    return Pdata
# print(Pdata)

#Ending the session. Every creation .route will have open and close
session.close()







[]


In [9]:
#Start a new session before starting. 
session = Session(engine)

#Working on the stations

#variable to capture the station
statcap = []

# Return a JSON list of stations from the dataset.
# @ClimateApp.route("/api/v1.0/stations")
def stations():
    statn = session.query(station.station, station.name, station.latitude, station.longitude, station.elevation).all()

    session.close()

    for station, station_name, latitude, longitude, elevation in station:
        ScapD = {}
        ScapD["station"] = station
        ScapD["name"] = station_name
        ScapD["latitude"] = latitude
        ScapD["elevation"] = elevation
        ScapD["longitude"] = longitude
        statcap.append(ScapD)
    # return jsonify(statcap)
    return statcap

session.close()


In [20]:
print(statcap)

[]


In [10]:
#create a new session, create the tobs route

#Like the precipitation route, this is mostly copy and paste from the starter. If done correctly.

session = Session(engine)
Tempcap = []
# @ClimateApp.route("/api/v1.0/tobs")
def tobs():
    #Get a count for all the stations activity
    ACstation = session.query(measurement.station, func.count(measurement.station)).\
        group_by(measurement.station).order_by( func.count(measurement.station).desc()).all()
    
    #Whats the most active and Least Active Station
    MactiveS = ACstation[0][0]
    LactiveS = ACstation[8][0]
    print(f"The most active station is " + MactiveS + ". ")
    print(f"The least active station is " + LactiveS + ". ")

    #Do a query to get the temperature Data of the most active station
    twelvemonths = dt.date(2017, 8, 23) - relativedelta(years=1)
    twelvemtemp = session.query(measurement.date,  measurement.tobs).filter(measurement.date >= twelvemonths).order_by(measurement.date.asc()).all()

    #Capture the data
    for date, temperature in twelvemtemp:
        TdCap = {}
        TdCap[date] = temperature
        Tempcap.append(TdCap)
    return jsonify(Tempcap)
    # return Pdata
session.close()



In [12]:
#   * Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.

#   * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.
session = Session(engine)
StartCap = []
@ClimateApp.route("/api/v1.0/<start>")

def begin(start):

    STDStats = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
        filter(measurement.date >- start).all
    







session.close()

In [23]:
#   * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` for dates between the start and end date inclusive.
startdate = session.query(measurement.date).order_by(measurement.date).first()
print(startdate)

('2010-01-01',)


In [29]:

alpha = session.query(measurement.date).order_by(measurement.date).first()
omega = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(alpha, omega)    

('2010-01-01',) ('2017-08-23',)


In [33]:

    STDA = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
        filter(measurement.date < omega).\
        filter(measurement.date > alpha).all()    

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 0 - probably unsupported type.
[SQL: SELECT min(measurement.tobs) AS min_1, max(measurement.tobs) AS max_1, avg(measurement.tobs) AS avg_1 
FROM measurement 
WHERE measurement.date < ? AND measurement.date > ?]
[parameters: (('2017-08-23',), ('2010-01-01',))]
(Background on this error at: http://sqlalche.me/e/13/rvf5)

In [9]:

if __name__ == '__main__':
    ClimateApp.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Restarting with windowsapi reloader


SystemExit: 2

SyntaxError: unexpected EOF while parsing (<ipython-input-101-548aa92d94d3>, line 12)